In [299]:
import pandas as pd
import numpy as np

In [300]:
data = pd.read_parquet('/Users/vittoriomanfriani/Desktop/bonds_us.pq')

In [301]:
data = data[:50000]
data.head()

price  yield  dv01  coupon   maturity
timestamp  id                                                      
2000-01-03 US912810BU17  100.812500  5.895   NaN   8.250 2005-05-15
           US912810BX55  101.625000  6.781   NaN   7.625 2007-02-15
           US912810BZ04  102.734375  6.803   NaN   7.875 2007-11-15
           US912810CC00  104.968750  6.797   NaN   8.375 2008-08-15
           US912810CE65  106.453125  6.817   NaN   8.750 2008-11-15

In [302]:
# Convert 'maturity' to datetime
data['maturity'] = pd.to_datetime(data['maturity'], errors='coerce')

# Drop rows where the 'maturity' is NaT (missing)
data = data.dropna(subset=['maturity'])

# Compute time to maturity
data.reset_index(inplace=True)
data['time to maturity'] = (data['maturity'] - data['timestamp']) / pd.Timedelta(days=365.25)

# Get time to maturities dataset
maturities = data.pivot(index='timestamp', columns='id', values='time to maturity')
maturities.head()

# Get yield dataset
yields = data.pivot(index='timestamp', columns='id', values='yield')

In [303]:
# Before proceeding we interpolate nans only if there is one consecutive

# Function to check single NaN in each column
def is_single_nan(series):
    mask = series.isna()
    # Single NaN is identified as a NaN surrounded by non-NaNs
    return mask & ~mask.shift(1, fill_value=False) & ~mask.shift(-1, fill_value=False)

# apply the function both to maturities and yields dataset

# Mask for single NaNs
single_nan_mask_maturities = maturities.apply(is_single_nan)
single_nan_mask_yields = yields.apply(is_single_nan)

maturities = maturities.where(~single_nan_mask_maturities, maturities.interpolate(method='linear', limit=1, axis=0))
yields = yields.where(~single_nan_mask_yields, yields.interpolate(method='linear', limit=1, axis=0))

In [313]:
# Apply Nelson-Siegel Model
def nelson_siegel(params, maturities, lambd):
    beta0, beta1, beta2 = params
    t = maturities
    alpha_1 = (1 - np.exp(-t/lambd))/(t/lambd)
    alpha_2 = (1 - np.exp(-t/lambd))/(t/lambd) - np.exp(-t/lambd)
    return beta0 + beta1 * alpha_1 + beta2 * alpha_2

In [305]:
# Error function to minimize to find optimal params
def error_function(params, maturities, data, lambd):
    data_hat = nelson_siegel(params, maturities, lambd)
    return np.sum((data - data_hat) ** 2)

# We define ridge error function as
def ridge_error_function(params, maturities, data, lambd, alpha=0.1):
    data_hat = nelson_siegel(params, maturities, lambd)
    error = np.sum((data - data_hat) ** 2) 
    regularization = alpha * (params[0]**2 + params[1]**2 + params[2]**2) 
    return error + regularization

In [311]:
# Apply Nelson-Siegel Model to the dataset
from scipy.optimize import minimize
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def apply_nelson_siegel(yields, maturities, lambdas = list(np.linspace(0.027, 1, 10)), ridge = False, alpha = 0.1):

    # Store results in a DataFrame
    fitted_results = []
    test_metrics = []
    
    initial_params = [0.03, -0.01, 0.01] 
    
    # Append arbitrary values to the list of possible lambdas 
    # (1.37 is the one that reflects best the curvature of 
    # the yield curve and 3 is the one that handles best multicollinearity)
    lambdas.append(1.37)
    lambdas.append(3)

    for i in range(yields.shape[0]):
        date = yields.index[i]
        current_yields = yields.iloc[i].dropna()
        current_maturities = maturities.iloc[i].dropna()
        
        # Align indices of current_yields and current_maturities
        valid_indices = current_yields.index.intersection(current_maturities.index)
        current_yields = current_yields.loc[valid_indices]
        current_maturities = current_maturities.loc[valid_indices]
        
        # Split data into train and test sets (80% train, 20% test)
        train_maturities, test_maturities, train_yields, test_yields = train_test_split(
            current_maturities, current_yields, test_size=0.2, random_state=42
        )
        
        # Initialize variables to store best parameters
        best_loss = float("inf")
        best_params = None
        best_lambda = None

        # Grid search over lambda
        for lambd in lambdas:
            
            if ridge == False:
                result = minimize(
                    error_function,
                    initial_params,
                    args=(train_maturities, train_yields, lambd),  
                    method="L-BFGS-B",
                    options={'maxiter': 1000}, 
                )
            
            if ridge == True:
                # Grid Search over lambdas
                result = minimize(
                    ridge_error_function,
                    initial_params,
                    args=(train_maturities, train_yields, lambd, alpha),
                    method="L-BFGS-B",
                    options={'maxiter': 1000}  
                )
            # Update best parameters and lambda if this result is better
            if result.fun < best_loss:
                best_loss = result.fun
                best_params = result.x
                best_lambda = lambd
        
        # Compute predictions on the test set
        test_predictions = nelson_siegel(best_params, test_maturities, best_lambda)
        
        # Compute R^2
        ss_res = np.sum((test_yields - test_predictions) ** 2)  
        ss_tot = np.sum((test_yields - np.mean(test_yields)) ** 2) 
        r_squared = 1 - (ss_res / ss_tot)
        
        # Compute Mean Squared Error
        mse = mean_squared_error(test_yields, test_predictions)

        # Store results for the current date
        fitted_results.append({
            "Date": date,
            "Beta0 (Level)": best_params[0],
            "Beta1 (Slope)": best_params[1],
            "Beta2 (Curvature)": best_params[2],
            "Lambda": best_lambda, 
        })
        
        test_metrics.append({
            "Date": date,
            "R^2": r_squared,
            "MSE": mse,
        })

    # Convert results to a DataFrame
    fitted_results_df = pd.DataFrame(fitted_results)
    test_metrics_df = pd.DataFrame(test_metrics)

    return fitted_results_df, test_metrics_df

In [335]:
# Optimization problems without grid search but treating lambda as another parameter to optimize

# Updated Nelson-Siegel model
def nelson_siegel_lambd(params, maturities):
    beta0, beta1, beta2, lambd = params
    
    # set a min value for lambda to account for 0 division in optimization problems
    lambd = max(lambd, 1e-6)
    
    t = maturities
    
    alpha_1 = (1 - np.exp(-t/lambd))/(t/lambd)
    alpha_2 = (1 - np.exp(-t/lambd))/(t/lambd) - np.exp(-t/lambd)
    return beta0 + beta1 * alpha_1 + beta2 * alpha_2

# Error function to minimize to find optimal params
def error_function_lambd(params, maturities, data):
    data_hat = nelson_siegel_lambd(params, maturities)
    return np.sum((data - data_hat) ** 2)

# We define ridge error function as
def ridge_error_function_lambd(params, maturities, data, alpha=0.1):
    data_hat = nelson_siegel_lambd(params, maturities)
    error = np.sum((data - data_hat) ** 2) 
    regularization = alpha * (params[0]**2 + params[1]**2 + params[2]**2) 
    return error + regularization


# Fit the model to the data and get performance metrics
def apply_nelson_siegel_lambd(yields, maturities, ridge = False, alpha = 0.1):

    # Store results in a DataFrame
    fitted_results = []
    test_metrics = []
    
    initial_params = [0.01, 0.01, 0.01, 1] 

    for i in range(yields.shape[0]):
        date = yields.index[i]
        current_yields = yields.iloc[i].dropna()
        current_maturities = maturities.iloc[i].dropna()
        
        # Align indices of current_yields and current_maturities
        valid_indices = current_yields.index.intersection(current_maturities.index)
        current_yields = current_yields.loc[valid_indices]
        current_maturities = current_maturities.loc[valid_indices]
        
        # Split data into train and test sets (80% train, 20% test)
        train_maturities, test_maturities, train_yields, test_yields = train_test_split(
            current_maturities, current_yields, test_size=0.2, random_state=42
        )
        if ridge == False:
            result = minimize(
                    error_function_lambd,
                    initial_params,
                    args=(train_maturities, train_yields),  
                    method="L-BFGS-B",
                    options={'maxiter': 1000}, 
                )
            
        if ridge == True:
            result = minimize(
                    ridge_error_function_lambd,
                    initial_params,
                    args=(train_maturities, train_yields, alpha),  
                    method="L-BFGS-B",
                    options={'maxiter': 1000}, 
                )
        
        params = result.x
        
        # Compute predictions on the test set
        test_predictions = nelson_siegel_lambd(params, test_maturities,)
        
        # Compute R^2
        ss_res = np.sum((test_yields - test_predictions) ** 2)  
        ss_tot = np.sum((test_yields - np.mean(test_yields)) ** 2) 
        r_squared = 1 - (ss_res / ss_tot)
        
        # Compute Mean Squared Error
        mse = mean_squared_error(test_yields, test_predictions)

        # Store results for the current date
        fitted_results.append({
            "Date": date,
            "Beta0 (Level)": params[0],
            "Beta1 (Slope)": params[1],
            "Beta2 (Curvature)": params[2],
            "Lambda": params[3], 
        })
        
        test_metrics.append({
            "Date": date,
            "R^2": r_squared,
            "MSE": mse,
        })

    # Convert results to a DataFrame
    fitted_results_df = pd.DataFrame(fitted_results)
    test_metrics_df = pd.DataFrame(test_metrics)

    return fitted_results_df, test_metrics_df

In [338]:
params, metrics = apply_nelson_siegel_lambd(yields, maturities, ridge=True)

In [339]:
metrics.mean()

Date    2000-07-28 21:15:42.281879168
R^2                          0.562883
MSE                          0.114856
dtype: object

In [314]:
params_ridge, metrics_ridge = apply_nelson_siegel(yields, maturities, ridge = True)

In [315]:
metrics_ridge.mean()

Date    2000-07-28 21:15:42.281879168
R^2                          0.557819
MSE                          0.113212
dtype: object

In [317]:
# clean the dataset of factors
factors_df = pd.DataFrame(index = params.Date)
factors_df['Beta0 (Level)'] = np.array(params['Beta0 (Level)'])
factors_df['Beta1 (Slope)'] = np.array(params['Beta1 (Slope)'])
factors_df['Beta2 (Curvature)'] = np.array(params['Beta2 (Curvature)'])

In [318]:
# clean the dataset of factors from ridge model
factors_df_ridge = pd.DataFrame(index = params.Date)
factors_df_ridge['Beta0 (Level)'] = np.array(params_ridge['Beta0 (Level)'])
factors_df_ridge['Beta1 (Slope)'] = np.array(params_ridge['Beta1 (Slope)'])
factors_df_ridge['Beta2 (Curvature)'] = np.array(params_ridge['Beta2 (Curvature)']) 

In [319]:
# Get dataset of returns

# First we get a dataset of prices
prices = data.pivot(index='timestamp', columns='id', values='price')

# Then we get a dataset of coupons
coupons = data.pivot(index='timestamp', columns='id', values='coupon')

# Before proceeding we interpolate nans only if there is one consecutive

# apply the function both to prices and coupons dataset

# Mask for single NaNs
single_nan_mask_prices = coupons.apply(is_single_nan)
single_nan_mask_coupons = prices.apply(is_single_nan)

prices = prices.where(~single_nan_mask_prices, prices.interpolate(method='linear', limit=1, axis=0))
coupons = coupons.where(~single_nan_mask_coupons, coupons.interpolate(method='linear', limit=1, axis=0))

# Function to compute returns
def compute_returns(prices, coupons):
    # Get Daily Coupons
    daily_coupons = coupons/365
    
    # compute returns with formula (R_(t, t+1) = P_(t+1) + c  - P_(t) / P(t))
    returns = (prices + daily_coupons - prices.shift(1))/prices.shift(1)
    
    return returns 

returns = compute_returns(prices,coupons)


In [320]:
# Align the factors dataset to the on of returns
factors_df = factors_df.iloc[1:]
factors_df_ridge = factors_df_ridge.iloc[1:]
returns = returns.iloc[1:]

In [321]:
import statsmodels.api as sm

def rolling_regression(data, factors_df, window_size=252):
    # Initialize data structures to store loadings
    loading_datasets = {factor: pd.DataFrame(index=data.index[window_size:], columns=data.columns) 
                        for factor in ['const'] + list(factors_df.columns)}
    #Initialize dataset to store % variance explained
    variance_explained = pd.DataFrame(index=data.index[window_size:], columns=data.columns)

    # Iterate over each asset (column in `data`)
    for col in data.columns:
        y = data[col]
        
        # Perform rolling window regression
        for i in range(window_size, len(data)):
            
            # Handle Nans
            if pd.isna(data.loc[data.index[i], col]):
                for factor in ['const'] + list(factors_df.columns):
                    loading_datasets[factor].loc[data.index[i], col] = np.nan
                continue
                    
            # Select rolling window data
            y_window = y.iloc[i - window_size:i].dropna()
            X_window = factors_df.iloc[i - window_size:i]
            X_window = sm.add_constant(X_window)

            # Handle Nans if window length is not enough to perform the regression
            if y_window.shape[0] < window_size * 0.5:
                for factor in ['const'] + list(factors_df.columns):
                    loading_datasets[factor].loc[data.index[i], col] = np.nan
                continue
                    
            X_window = X_window.loc[y_window.index]
                    
            # Perform regression
            model = sm.OLS(y_window, X_window).fit()
            
            
            # Compute variance explained (R^2 as a percentage)
            y_pred = model.fittedvalues
            ss_res = np.sum((y_window - y_pred) ** 2)  # Residual Sum of Squares
            ss_tot = np.sum((y_window - np.mean(y_window)) ** 2)  # Total Sum of Squares
            r_squared = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
            variance_explained.loc[data.index[i], col] = r_squared * 100
            
            # Store coefficients for each factor
            for factor, loading in model.params.items():
                loading_datasets[factor].loc[data.index[i], col] = loading

    return loading_datasets, variance_explained

In [322]:
loading_datasets, variance_explained = rolling_regression(returns, factors_df)

In [323]:
loading_datasets_ridge, variance_explained_ridge = rolling_regression(returns, factors_df_ridge)

In [266]:
def factor_and_idio_returns(returns, loading_datasets):
    
    # get columns names
    names = list(loading_datasets.keys())[1:]
    
    factor_returns = pd.DataFrame(index=loading_datasets[names[0]].index, columns=loading_datasets[names[0]].columns)
    idio_returns = pd.DataFrame(index=loading_datasets[names[0]].index, columns=loading_datasets[names[0]].columns)
    factor_returns.fillna(0, inplace=True)
    idio_returns.fillna(0, inplace=True)
    
    # align the returns dataset
    returns = returns.loc[factor_returns.index]

    for name in names:
        factor_returns += loading_datasets[name] * returns
    
    idio_returns = returns - factor_returns
    
    return factor_returns, idio_returns

In [324]:
factor_returns, idio_returns = factor_and_idio_returns(returns, loading_datasets)

In [325]:
factor_returns_ridge, idio_returns_ridge = factor_and_idio_returns(returns, loading_datasets_ridge)